<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_E1_Cassandra_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Install Cassandra

install https://phoenixnap.com/kb/install-cassandra-on-ubuntu

In [1]:
!apt update > null
!apt install openjdk-8-jdk -y > null
#!java -version
!apt install apt-transport-https > null

In [2]:
!sh -c 'echo "deb http://www.apache.org/dist/cassandra/debian 40x main" > /etc/apt/sources.list.d/cassandra.list'
!wget -q -O - https://www.apache.org/dist/cassandra/KEYS | sudo apt-key add -

OK


In [3]:
!apt update > null
!apt install cassandra > null

## Start the Service

In [4]:
#!service cassandra stop
# the ulimit error does not matter
!service cassandra restart

 * Restarting Cassandra cassandra
/etc/init.d/cassandra: 72: ulimit: error setting limit (Operation not permitted)
   ...done.


In [7]:
# keep checking service status until you see [+] next to cassandra
# then wait for 30 seconds MORE
!service  --status-all 

 [ + ]  cassandra
 [ - ]  dbus
 [ ? ]  hwclock.sh
 [ ? ]  kmod
 [ - ]  ntp
 [ - ]  procps
 [ - ]  rsync
 [ + ]  udev
 [ - ]  x11-common


In [9]:
# this will give an error if the cassandra service has not started
# Wait for some time for the service to start
# If required reboot system or stop, restart cassandra service
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address    Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  127.0.0.1  69.09 KiB  16      100.0%            81d06b18-a09d-41f0-a39f-3031a804ba2d  rack1



In [ ]:
# basic test
#!cqlsh

# Basic Commands 
how to execute commands through scripts https://stackoverflow.com/questions/25286253/executing-cql-through-shell-script <br>
some examples https://techfossguru.com/apache-cassandra-python-step-step-guide-ubuntu-example/

In [10]:
!cqlsh -e "help" 


Documented shell commands:
CAPTURE  CLS          COPY  DESCRIBE  EXPAND  LOGIN   SERIAL  SOURCE   UNICODE
CLEAR    CONSISTENCY  DESC  EXIT      HELP    PAGING  SHOW    TRACING

CQL help topics:
AGGREGATES               CREATE_KEYSPACE           DROP_TRIGGER      TEXT     
ALTER_KEYSPACE           CREATE_MATERIALIZED_VIEW  DROP_TYPE         TIME     
ALTER_MATERIALIZED_VIEW  CREATE_ROLE               DROP_USER         TIMESTAMP
ALTER_TABLE              CREATE_TABLE              FUNCTIONS         TRUNCATE 
ALTER_TYPE               CREATE_TRIGGER            GRANT             TYPES    
ALTER_USER               CREATE_TYPE               INSERT            UPDATE   
APPLY                    CREATE_USER               INSERT_JSON       USE      
ASCII                    DATE                      INT               UUID     
BATCH                    DELETE                    JSON            
BEGIN                    DROP_AGGREGATE            KEYWORDS        
BLOB                     DROP_COLUMNF

In [11]:
!cqlsh -e "CREATE KEYSPACE praxis WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 };"


Warnings :
Your replication factor 3 for keyspace praxis is higher than the number of nodes 1



In [12]:
# since we are not specifying keyspace separately, we have to prefix table name with keyspace
!cqlsh -e "CREATE TABLE praxis.users (user_id int PRIMARY KEY, first_name text, last_name text );"

In [13]:
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (1, 'Prithwis', 'Mukerjee');"
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (2, 'Sanniti', 'Chatterjee');"
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (3, 'Amrita', 'Ghosh');"

In [14]:
!cqlsh -e "select * from praxis.users;"


 user_id | first_name | last_name
---------+------------+------------
       1 |   Prithwis |   Mukerjee
       2 |    Sanniti | Chatterjee
       3 |     Amrita |      Ghosh

(3 rows)


In [15]:
#another way to specifiy keyspace
!cqlsh -e "use praxis;\
           select * from users;"


 user_id | first_name | last_name
---------+------------+------------
       1 |   Prithwis |   Mukerjee
       2 |    Sanniti | Chatterjee
       3 |     Amrita |      Ghosh

(3 rows)


In [32]:
# easier way to specify keyspace, similar specifying database in mysql
!cqlsh -k praxis -e "select * from users;"


 user_id | first_name  | last_name
---------+-------------+------------
       1 | Dr Prithwis |   Mukerjee
       2 |     Sanniti | Chatterjee
       4 | Charanpreet |      Singh
       3 |      Amrita |      Ghosh

(4 rows)


# Python
https://docs.datastax.com/en/developer/python-driver/3.25/

In [17]:
!pip install cassandra-driver

     |████████████████████████████████| 3.8 MB 4.1 MB/s 


In [18]:
from cassandra.cluster import Cluster

In [19]:
cluster = Cluster(['127.0.0.1'],port=9042)

In [20]:
session = cluster.connect('praxis',wait_for_all_pools=True)

In [21]:
session.execute('USE praxis')

In [22]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Prithwis Mukerjee
2 Sanniti Chatterjee
3 Amrita Ghosh


In [23]:
session.execute("INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (4, 'Charanpreet', 'Singh');")

In [24]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Prithwis Mukerjee
2 Sanniti Chatterjee
4 Charanpreet Singh
3 Amrita Ghosh


In [30]:
session.execute("update praxis.users set first_name = 'Dr Prithwis' where user_id = 1;")

In [31]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Dr Prithwis Mukerjee
2 Sanniti Chatterjee
4 Charanpreet Singh
3 Amrita Ghosh


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)